In [1]:
path_code_dir = "/home/cbisot/pycode/MscThesis"
import sys  
sys.path.insert(0, path_code_dir)
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path
from amftrack.pipeline.functions.node_id import (
    second_identification,
)
from amftrack.pipeline.functions.extract_graph import (
    from_nx_to_tab,
)
import scipy.io as sio
import pickle
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project, run_parallel_stitch
from amftrack.notebooks.analysis.data_info import *
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path, get_plate_number, get_postion_number,get_begin_index
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from amftrack.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children
from collections import Counter
from IPython.display import clear_output
from amftrack.notebooks.analysis.data_info import *
from amftrack.pipeline.functions.node_id import reconnect_degree_2



In [2]:
path_code_dir = "/home/cbisot/pycode/MscThesis"

import sys

sys.path.insert(0, path_code_dir)
from scipy import sparse
from pymatreader import read_mat

from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path
from amftrack.pipeline.functions.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
from amftrack.pipeline.functions.node_id import remove_spurs

import scipy.sparse
import pickle

directory = directory_project
plate = 38
i = 150
dates_datetime = get_dates_datetime(directory, plate)
dates_datetime.sort()
date_datetime = dates_datetime[i]
date = date_datetime
directory_name = get_dirname(date, plate)
path_snap = directory + directory_name
skel = read_mat(path_snap + "/Analysis/skeleton_pruned_realigned.mat")["skeleton"]
skeleton = scipy.sparse.dok_matrix(skel)

# nx_graph_poss=[generate_nx_graph(from_sparse_to_graph(skeleton)) for skeleton in skels_aligned]
# nx_graphs_aligned=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
# poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]
# nx_graph_pruned=[clean_degree_4(prune_graph(nx_graph),poss_aligned[i])[0] for i,nx_graph in enumerate(nx_graphs_aligned)]
nx_graph, pos = generate_nx_graph(from_sparse_to_graph(skeleton))
# Optional, to remove spurs
print("removing_spurs")
nx_graph, pos = remove_spurs(nx_graph, pos)

removing_spurs


In [16]:
exp2 = get_exp((38, 131, 131 + 20), directory_project)

exp2.load_compressed_skel()

In [40]:
spurs = []
t = 10
nx_graph_tm1 = exp2.nx_graph[t]
pos_tm1 = exp2.positions[t]
for edge in nx_graph_tm1.edges:
    edge_data = nx_graph_tm1.get_edge_data(*edge)
    if (
        nx_graph_tm1.degree(edge[0]) == 1 or nx_graph_tm1.degree(edge[1]) == 1
    ) and edge_data["weight"] < 60:
        spurs.append(edge)

In [45]:
len(spurs), len(nx_graph_tm1.edges)

(50, 169)

In [49]:
spur = choice(spurs)
plt.close("all")
plot_raw_plus(exp2, t, [spur[0], spur[1]])

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
exp2.plot([t], [[spur[0], spur[1]]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
spurs = []
for spur in spurs:
    nx_graph_tm1.remove_edge(spur[0], spur[1])
reconnect_degree_2(nx_graph_tm1, pos_tm1)

In [47]:
for edge in nx_graph_tm1.edges:
    edge_data = nx_graph_tm1.get_edge_data(*edge)
    if (
        nx_graph_tm1.degree(edge[0]) == 1 or nx_graph_tm1.degree(edge[1]) == 1
    ) and edge_data["weight"] < 60:
        spurs.append(edge)

In [7]:
def remove_spurs(nx_g, pos):
    found = True
    while found:
        spurs = []
        found = False
        for edge in nx_g.edges:
            edge_data = nx_g.get_edge_data(*edge)
            if (nx_g.degree(edge[0]) == 1 or nx_g.degree(edge[1]) == 1) and edge_data[
                "weight"
            ] < 60:
                spurs.append(edge)
                found = True
        for spur in spurs:
            nx_g.remove_edge(spur[0], spur[1])
        reconnect_degree_2(nx_g, pos, has_width=False)
    return (nx_g, pos)

In [8]:
nx_graph, pos = remove_spurs(nx_graph, pos)

In [55]:
t = 12
nx_graph_tm1 = exp2.nx_graph[t]
pos_tm1 = exp2.positions[t]
nx, po = remove_spurs(nx_graph_tm1, pos_tm1)

In [56]:
spurs = []
t = 10
nx_graph_tm1 = nx
pos_tm1 = po
for edge in nx_graph_tm1.edges:
    edge_data = nx_graph_tm1.get_edge_data(*edge)
    if (
        nx_graph_tm1.degree(edge[0]) == 1 or nx_graph_tm1.degree(edge[1]) == 1
    ) and edge_data["weight"] < 60:
        spurs.append(edge)

In [57]:
spurs

[]